# Metoda `apply`

Metoda `apply()` nám umožňuje používat vlastní funkce. Pokud tedy nějakou úpravu dat sám modul `pandas` neumí, případně pokud by bylo jejich použití příliš složité a nepřehledné, můžeme si vytvořit vlastní funkci.

Vlastní funkci vytváříme pomocí klíčového slova `def`. Poté přidáme název funkce a parametry, které příjmá. U `pandas` máme možnost použít funkci dvěma způsoby:

- pracovat s hodnotou v konkrétním sloupci,
- pracovat se všemi hodnotami v řádku.

Vyzkoušíme si použití pro celou tabulku. V tabulce [vysledky.csv](vysledky.csv) máme známky z maturitní zkoušky a naším úkolem je zjistit, zda člověk prospěl s vyznamenáním, prospěl (bez vyznamenání) či neprospěl. Pravidla jsou následující:

- pokud má člověk průměr do 1.5 a nemá horší známku než 2, prospěl s vyznamenáním,
- pokud dostal známku 5, pak neprospěl,
- ve všech ostatních případech prospěl.

Vytvoříme si funkci `evaluate_result()`, která bude mít jako jeden parametr `row` (řádek tabulky). Parametr `row()` je série, tj. můžeme vybírat konkrétní hodnoty (např. známku z vybraného předmětu), dále můžeme využívat agregační funkce jako průměr či maximum.

In [6]:
import pandas

def evaluate_result(row):
    row = row.iloc[2:]
    if row.mean() <= 1.5 and row.max() <= 2:
        return "Prospěl(a) s vyznamenáním"
    elif row.max() == 5:
        return "Neprospěl(a)"
    else:
        return "Prospěl(a)"

data = pandas.read_csv("vysledky.csv")
data["vysledek"] = data.apply(evaluate_result, axis=1)
data.head()

,Jméno,Poplatek,Český jazyk,Anglický jazyk,Informatika,Matematika,vysledek
0,Mirek Dušín,1.4.2023,2,2,1,1,Prospěl(a) s vyznamenáním
1,Jarka Metelka,3.4.2023,3,5,3,1,Neprospěl(a)
2,Jindra Hojer,7.4.2023,2,2,1,3,Prospěl(a)
3,Červenáček,NaN,1,1,1,4,Prospěl(a)
4,Rychlonožka,NaN,4,3,2,4,Prospěl(a)


Nyní si vyzkoušíme, jak funguje výběr jedné hodnoty nebo více hodnot. Uvažujme například, že nyní řešíme přijetí studentů/studentek na vysokou školu s matematickým zaměřením. Jako první zkontrolujeme, zda student zaplatil administrativní poplatek za podání přihlášky. Ve sloupci `Poplatek` vidíme datum (pokud je poplatek zaplacen) nebo prázdnou hodnotu (pokud zaplacen není). Na začátku tedy využijeme funkce `pandas.isnull()`, která vrátí `True`, pokud je pro daný řádek daný sloupec prázdný, a `False`, pokud pro daný řádek ve sloupci nějaká hodnota je. Dále řešíme, zda má být student přijet bez přijímací zkoušky. Škola umožňuje přijetí studentů bez přijímací zkoušky, pokud mají z maturity jedničku z matematiky a současně mají průměr všech známek menší nebo roven 2.

In [7]:
def evaluate_application(row):
    row = row.iloc[1:]
    if pandas.isnull(row["Poplatek"]):
        return "Vyřazen - nezaplatil"
    elif row["Matematika"] == 1 and row["Český jazyk":"Matematika"].mean() <= 2:
        return "Přijat bez PZ"
    else:
        return "Musí absolvovat PZ"
    
data["prijimaci_zkouska"] = data.apply(evaluate_application, axis=1)
data.head()

,Jméno,Poplatek,Český jazyk,Anglický jazyk,Informatika,Matematika,vysledek,prijimaci_zkouska
0,Mirek Dušín,1.4.2023,2,2,1,1,Prospěl(a) s vyznamenáním,Přijat bez PZ
1,Jarka Metelka,3.4.2023,3,5,3,1,Neprospěl(a),Musí absolvovat PZ
2,Jindra Hojer,7.4.2023,2,2,1,3,Prospěl(a),Musí absolvovat PZ
3,Červenáček,NaN,1,1,1,4,Prospěl(a),Vyřazen - nezaplatil
4,Rychlonožka,NaN,4,3,2,4,Prospěl(a),Vyřazen - nezaplatil


## Čtení na doma - volání s parametry

Metoda `apply()` umožňuje přidání vlastních hodnot při volání funkce. Pokud bychom například chtěli změnit hranici pro přijetí studenta (studentky) mezi roky, namísto úpravy vnitřku funkce můžeme přidat parametr `hranice_prumer`. Jeho konkrétní hodnotu pak zadáme jako parametr `args`. Protože obecně můžeme mít parametrů více, zapisujeme jejich hodnoty do seznamu.

In [8]:
def evaluate_application(row, hranice_prumer):
    row = row.iloc[1:]
    if pandas.isnull(row["Poplatek"]):
        return "Vyřazen - nezaplatil"
    elif row["Matematika"] == 1 and row["Český jazyk":"Matematika"].mean() <= hranice_prumer:
        return "Přijat bez PZ"
    else:
        return "Pozvat na PZ"
    
data["prijimaci_zkouska"] = data.apply(evaluate_application, axis=1, args=[1.5])
data.head()

,Jméno,Poplatek,Český jazyk,Anglický jazyk,Informatika,Matematika,vysledek,prijimaci_zkouska
0,Mirek Dušín,1.4.2023,2,2,1,1,Prospěl(a) s vyznamenáním,Přijat bez PZ
1,Jarka Metelka,3.4.2023,3,5,3,1,Neprospěl(a),Pozvat na PZ
2,Jindra Hojer,7.4.2023,2,2,1,3,Prospěl(a),Pozvat na PZ
3,Červenáček,NaN,1,1,1,4,Prospěl(a),Vyřazen - nezaplatil
4,Rychlonožka,NaN,4,3,2,4,Prospěl(a),Vyřazen - nezaplatil
